In [1]:
import contextlib


def make_val_and_grad_fn(value_fn):
  @functools.wraps(value_fn)
  def val_and_grad(x):
    return tfp.math.value_and_gradient(value_fn, x)
  return val_and_grad



@contextlib.contextmanager
def timed_execution():
  t0 = time.time()
  yield
  dt = time.time() - t0
  print('Evaluation took: %f seconds' % dt)


def np_value(tensor):
  """Get numpy value out of possibly nested tuple of tensors."""
  if isinstance(tensor, tuple):
    return type(tensor)(*(np_value(t) for t in tensor))
  else:
    return tensor.numpy()

def run(optimizer):
  """Run an optimizer and measure it's evaluation time."""
  optimizer()  # Warmup.
  with timed_execution():
    result = optimizer()
  return np_value(result)

In [2]:
import numpy as np
from math import gamma
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import time
import pandas as pd

import os
os.chdir("C:\\Users\\user\\Downloads\\Thesis_paper_1\\calibration")

np.set_printoptions(suppress=True) # supress scientific notation

import matplotlib.ticker as mtick
import locale
locale.setlocale(locale.LC_NUMERIC, "de_DE")


plt.rcParams['axes.formatter.use_locale'] = True



pd.set_option('display.max_columns', 50) 

In [3]:
%notebook "C:\Users\user\Downloads\Thesis_paper_1\calibration\model_tensor.ipynb"

In [4]:

import tensorflow as tf
import tensorflow_probability as tfp

import functools

from scipy.optimize import differential_evolution
from scipy.optimize import basinhopping
from scipy.optimize import shgo

In [5]:
## parameters



def get_data(year):
    w='data_' + str(year)
    alfa = tf.convert_to_tensor(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,5], dtype='float64').reshape(1,3) )
    phi = tf.convert_to_tensor(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,4], dtype='float64').reshape(1,3) )
    return alfa, phi



def pars(year):
    global alfa, rho, beta, theta, eta, psi, kappa, lamb, gamma1, G, phi, s
    rho = tf.constant(0.19, tf.float64)
    beta = tf.constant(0.231, tf.float64)
    theta = tf.constant(2.0, tf.float64)
    eta = tf.constant(0.103, tf.float64)
    psi = eta**eta*(1.0-eta)**(1.0-eta)
    kappa = tf.divide(1.0, (1.0 - eta))
    lamb = 1 - tf.divide(kappa, theta)
    gamma1 = tf.multiply(eta, kappa)    
    G = tf.constant(gamma(1 - tf.divide(kappa, tf.multiply(theta, (1-rho)) ) ), tf.float64  )      
    alfa, phi = get_data(year)
    s = tf.pow( (1 + tf.divide( (1-eta), ( tf.multiply(beta, phi) ) ) ), -1 )


pars(15)

In [6]:

df = pd.DataFrame()

l = [92, 95, 2, 15]

for i in l:
    w='data_' + str(i)
    k ='alfa_' + str(i)
    df[k] = np.array(pd.read_excel(f'{w}.xlsx').iloc[:,5], dtype='float64').reshape(3)



df

In [7]:
df2 = pd.DataFrame()


for i in l:
    w='data_' + str(i)
    j ='phi_' + str(i)
    df2[j] = np.array(pd.read_excel(f'{w}.xlsx').iloc[:,4], dtype='float64').reshape(3)

    
df2

In [8]:
def s_f(phi):
    return tf.pow( (1 + tf.divide( (1-eta), ( tf.multiply(beta, phi) ) ) ), -1 )

In [9]:
# taus



def taus(st):
    tau_h = tf.random.uniform(maxval=10, minval=-10, shape=[1, st], dtype='float64')
    x = np.array([tau_h])
    x = tf.convert_to_tensor(x, tf.float64)
    return x
    

In [10]:
def N_if(x1):
    S = tf.pow(s, phi*(theta*lamb+ kappa) )
    S2 =tf.pow((1 - s), ( (theta*lamb)/(beta*kappa) ) )
    H = tf.pow((1 + x1[0]), (theta*lamb + kappa))    
    C = tf.multiply(tf.multiply(tf.pow(psi, theta*lamb), eta**gamma1 ), G )
    N_i = tf.multiply(S, tf.multiply(S2, tf.multiply(H, C)) )
    return N_i

In [11]:

#  linear system 

def matrix():
    global A, B
    A = np.repeat(alfa, st).reshape(st, st).T       # Matriz de parâmetros    
    B = np.repeat(alfa, st).reshape(st, st).T        # Matriz de alfas 
    for i in range(st):
        for j in range(st):
            if i==j:
                A[i, j] = 1 - (A[i, j] - 1)*(theta*lamb + gamma1)
                B[i, j] = B[i, j] - 1
            if i != j:
                A[i, j] = -A[i, j]*(theta*lamb + gamma1) 
    A = tf.convert_to_tensor(A, tf.float64)
    B = tf.convert_to_tensor(B, tf.float64)
    

def lin_sys(x1):
    #global N_i
    #print(f'x1 = {x1}')
    N_i = N_if(x1)
    N_i2 = tf.math.log( N_i )                                                    # Logaritmo de N_i
    C = tf.math.log(alfa)                                                        # Logaritmo dos alfas
    D = tf.reshape(tf.tensordot(B, tf.reshape(N_i2, (3,1)) , 1), (1, 3) ) + C    # produto matricial                     
    res = tf.reshape(tf.exp(tf.linalg.solve(A, tf.reshape(D, (3,1)) ) ), (1,3) )  # Resolvendo o sistema  A w = D
    #res =tf.reshape(res, (1,3))
    return res

In [12]:
def w_til_f(x1):
    S = tf.pow(s, phi)
    S2 = tf.pow( (1 - s), 1/(beta*kappa) )
    H = 1 + x1[0]
    w_til = tf.multiply(psi, tf.multiply(H, tf.multiply(S, tf.multiply(w , S2))))
    return w_til

   

In [13]:
# wages - model 

def W_i_f(x1):
    w_til = w_til_f(x1)
    sum_wtil = tf.pow( tf.reduce_sum( tf.pow(w_til, theta) ) , (kappa/theta) ) 
    S = tf.pow((1-s), (-1/beta))
    p_i = tf.pow(w_til, theta)/ tf.reduce_sum( tf.pow(w_til, theta) )   
    W_i = tf.multiply(kappa, tf.multiply(G, tf.multiply(S, tf.multiply(sum_wtil, p_i)) ) )    
    return W_i, p_i

In [14]:
# human capital

def hc_f(x1):
    W = tf.pow(w , (theta*lamb + gamma1) )
    W_til = tf.pow(tf.reduce_sum(tf.pow(w_til_f(x1), theta) ), lamb)
    N_i = N_if(x1)
    Hc = tf.multiply(N_i, tf.divide(W, W_til))
    return Hc

In [15]:
# PNAD - DATA


def get_p_i(year):
    global p_s2
    w='data_' + str(year)
    p_i = tf.convert_to_tensor(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,2]).reshape(1,3), tf.float64)
    p_s2 = p_i[1:st]
    return p_i


def get_W_i(year):
    p_i = get_p_i(year)
    global W_id
    w='data_' + str(year)
    W_id = tf.convert_to_tensor(np.multiply(np.array(pd.read_excel(f'{w}.xlsx').iloc[:,1]), p_i).reshape(1,3), tf.float64)
    return W_id

In [16]:
#@tf.function
def tflow(x1):    
    global w     
    w = lin_sys(x1)
    x1 = tf.reshape(x1, (1, 3))
    tf.convert_to_tensor(x1, tf.float64)
    W_im = W_i_f(x1)[0]    
    D = tf.reduce_sum(tf.pow( tf.divide(tf.math.squared_difference(W_id, W_im), W_id ) , 2)) 
    #D = tf.math.log(D)
    return D

In [17]:
st=3
x1 = taus(st)
ano = 15
pars(15)
matrix()
get_W_i(ano)
get_p_i(ano)

tflow(taus(3))

In [18]:
# bounds


Bd =   ((-10.0, 10.0), )*st 

#Bd = (tf.constant(-20.0, dtype=tf.float64), tf.constant(20.0, dtype=tf.float64), )*st

Bd = np.array(Bd, dtype=object)

In [19]:
# callback


cc = 0
def callback(x):
    global cc
    cc += 1
    fobj = tflow(x)
    print(f'\033[1;033mObjetivo: {np.around(fobj, 10)}, iter: {cc}') 

In [20]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0



dtype= tf.float64
start = tf.constant(np.array([1.0, 3.10, -1.20]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.3,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [21]:
st = 3
ano= 15
x1 = taus(st)
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0



dtype= tf.float64
start = tf.constant(np.array([1.0, 3.10, -1.20]), dtype=dtype)


def nelder_mead():
  return tfp.optimizer.nelder_mead_minimize(
  tflow,
  initial_vertex=start,
  parallel_iterations =100,
  shrinkage=0.3,
  func_tolerance=0,
  position_tolerance=0.9e-14)

In [22]:
st = 3
x1 = taus(st)
ano= 15
pars(ano)
matrix()
get_W_i(ano)
get_p_i(ano)
cc=0

In [23]:
sol = differential_evolution(tflow, Bd, strategy='best1exp',
                             tol=0,
                             polish=True, 
                             popsize=100, 
                             maxiter=400,
                             recombination =0.7,
                             disp=True)

In [24]:

print(10*'--'+ 'Distorções calibradas' + 10*'--')
print(60*'*')
print(f'Objective function: {sol.fun}' )
print(f'Distortions: {sol.x.reshape(1,3)[0]}')




#-----------------  (-10, 10)

# 9.37196472 8.44134184 8.66419531 ----> 15

In [25]:
d = sol.x.reshape(1, st)

p_m = W_i_f(d)[1]
p_d = get_p_i(ano)

W_m = W_i_f(d)[0]
W_d = get_W_i(ano)


fig,ax = plt.subplots(1, 2, figsize=(14,8))
ax[0].scatter(p_m, p_d, s=100, color='red')
ax[0].plot([0.0, 0.7], [0.0, 0.7], 'k-', lw=2, label='linha de 45°', color='blue')
ax[0].grid(True)
ax[0].set_title(r'$p_i$', fontsize=20)
ax[0].tick_params(axis='both', which='major', labelsize=20)

ax[1].scatter(W_m, W_d, s=100, color='red')
ax[1].plot([np.min(W_m), np.max(W_m)], [np.min(W_m), np.max(W_m)], 'k-', lw=2, label='linha de 45°', color='blue')
ax[1].grid(True)
ax[1].set_title(r'$W_i$', fontsize=20)
ax[1].tick_params(axis='both', which='major', labelsize=20)

In [26]:
d = sol.x.reshape(1, st)


h = hc_f(d)

In [27]:
cd = 2
np.round(lin_sys(d), cd) 

In [28]:
np.round((alfa/h)*np.prod(np.power(h, alfa) ), cd)

In [29]:



df3 = pd.DataFrame()

l = [92, 95, 2, 15]

for i in l:
    pars(i)
    #w='data_' + str(i)
    k ='hc_' + str(i)
    df3[k] = np.array(hc_f(d)).reshape(3)


df3.index = ['AGR', 'IND', 'SERV']

In [30]:
df3

In [31]:
anos = [92, 95, '02', 15]
up = 0.04

fig, ax = plt.subplots(2, 2, figsize=(16,16))
x = [ax[0,0], ax[0,1], ax[1,0], ax[1,1]]

for i in range(len(x)):
    a = df3.iloc[:,i].keys()
    b = df3.iloc[:,i].values
    x[i].bar(a, b, color ='cyan', width = 0.2)
    x[i].set_title('Capital Humano'+ f' {anos[i]}', fontsize=20)
    x[i].tick_params(axis='both', which='major', labelsize=20)
    x[i].grid(b=True, which='major', color='k', linestyle=':', lw=.5, zorder=1)
    for xi, yi, l in zip(*[a, b, list(map(str, np.round(b,2)))] ):
        x[i].text(xi, yi + up, l,
                  horizontalalignment='center',
                #bbox=dict(facecolor='w', edgecolor='w', alpha=.5),
               fontsize=16)
    plt.tight_layout

    

In [32]:
# pegar os anos de estudo da pnad

def get_est(year):
    w='data_' + str(year)
    est = np.array(pd.read_excel(f'{w}.xlsx').iloc[:,3], dtype='float64').reshape(3) 
    return est

In [33]:
# calcular o capital humano do modelo

def calc_hc(year, d):
    pars(year)
    return np.array(hc_f(d)).reshape(3)

In [34]:
# cada ponto de dispersão é um ano 
# cada gráfico é de um setor

sector = ['Agropecuária', 'Indústria', 'Serviços']
anos = [92, 95, 2, 15]
n=len(anos)    # quantidade de anos
fig, ax = plt.subplots(3, 1, figsize=(8,16))

for j in range(3):
    ax[j].scatter([[calc_hc(i, d) for i in anos][k][j] for k in range(n) ] ,
                  [[get_est(i) for i in anos][k][j] for k in range(n) ] , marker='o', s=200, color='maroon' )
    ax[j].tick_params(axis='both', which='major', labelsize=20)
    ax[j].set_xlabel('Capital humano - Modelo', fontsize=20)
    ax[j].set_ylabel('Anos de estudo - PNAD', fontsize=20)
    ax[j].grid(b=True, which='major', color='k', linestyle=':', lw=.5)
    ax[j].set_title(f'{sector[j]}', fontsize=20)
plt.tight_layout()

In [35]:
# calcula o produto da economia

def pib(x1, year):
    Y = tf.math.reduce_prod(tf.pow(hc_f(x1), get_data(year)[0] ))
    return Y

    

In [36]:
# pega o pib dos dados do ipea

def pib_dados(year):
    da = pd.read_excel('IPEA_PIBS.xls', sheet_name='s2').iloc[:,[9,10]]
    da.set_index('index', inplace=True)
    return np.array(np.log(da.loc[[year]] ) ).reshape(1)

In [37]:
# cada ponto de dispersão é um ano diferente
# aqui uso o loop dentro da lista

anos = [92, 95, 2, 15]


fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter([pib_dados(i) for i in anos] , 
            [pib(d, i) for i in anos], marker='o', s=200, color='maroon' )
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('PIB - Dados', fontsize=20)
ax.set_ylabel('PIB - Modelo', fontsize=20)

In [38]:
anos = [92, 95, 2, 15]
sector = ['Agropecuária', 'Indústria', 'Serviços']
up = 0.01


fig, ax = plt.subplots(2, 2, figsize=(16,16))
x = [ax[0,0], ax[0,1], ax[1,0], ax[1,1]]


for i in range(len(x)):
    pars(anos[i])
    b = np.array(lin_sys(d) ).reshape(3)
    x[i].bar(sector, b, color ='gold', width = 0.2)
    x[i].set_title('Salário'+ f' {anos[i]}', fontsize=20)
    x[i].tick_params(axis='both', which='major', labelsize=20)
    x[i].grid(b=True, which='major', color='k', linestyle=':', lw=.5, zorder=1)
    for xi, yi, l in zip(*[sector, b, list(map(str, np.round(b,3)))] ):
        x[i].text(xi, yi+up, l,
                #bbox=dict(facecolor='w', edgecolor='w', alpha=.8),
                  horizontalalignment='center',
               fontsize=16)
    plt.tight_layout

In [39]:

df.describe()

In [40]:
%notebook "C:\Users\user\Downloads\Thesis_paper_1\calibration\model_tensor.ipynb"

In [41]:

#"C:\\Users\\user\\Downloads\\Thesis_paper_1\\dados"

pd.read_excel('C:\Users\user\Downloads\Thesis_paper_1\dados\prod_trab.xlsx', sheet_name='prod_hr')

In [42]:


pd.read_excel('C:\Users\user\Downloads\Thesis_paper_1\dados\prod_trab.xlsx', sheet_name='prod_hr')

In [43]:


pd.read_excel(r'C:\Users\user\Downloads\Thesis_paper_1\dados\prod_trab.xlsx', sheet_name='prod_hr')

In [44]:

ibre = pd.read_excel(r'C:\Users\user\Downloads\Thesis_paper_1\dados\prod_trab.xlsx', sheet_name='prod_hr')

In [45]:
df.index= df['Anos']
#df.drop('Anos', axis=1, inplace=True) 

In [46]:
ibre.index= df['Anos']
#df.drop('Anos', axis=1, inplace=True) 

In [47]:
ibre.index= ibre['Anos']
#df.drop('Anos', axis=1, inplace=True) 

In [48]:
ibre.loc[[1995, 2002, 2015], ['Agropecuária', 'Industria Total', 'Serviços Total']]

In [49]:
ibre.loc[[1995, 2002, 2015], ['Agropecuária', 'Industria Total', 'Serviços Total']].T

In [50]:
ibre2 = ibre.loc[[1995, 2002, 2015], ['Agropecuária', 'Industria Total', 'Serviços Total']].T

In [51]:
ibre2

In [52]:
ibre2 = ibre.loc[[1995, 2002, 2015], ['Agropecuária', 'Industria Total', 'Serviços Total']].T
ibre2

In [53]:
ibre.index = ['AGR', 'IND', 'SERV']

In [54]:
ibre2.index = ['AGR', 'IND', 'SERV']

In [55]:
ibre2

In [56]:
ibre2.index = ['AGR', 'IND', 'SERV']
ibre2.rename(columns={'1995':'95'}, inplace=True)

In [57]:
ibre2

In [58]:
ibre2

In [59]:
ibre2.index = ['AGR', 'IND', 'SERV']
ibre2.rename(columns={'1995':'95'}, inplace=True)

In [60]:
ibre2

In [61]:
ibre2.index = ['AGR', 'IND', 'SERV']
ibre2.rename(columns={'95':'1995'}, inplace=True)

In [62]:
ibre2

In [63]:
ibre2.columns

In [64]:
ibre2.columns[1]

In [65]:
ibre2.columns

In [66]:
ibre2.columns = ['95', '2', '15']

In [67]:
ibre2

In [68]:
df3

In [69]:
df3[::-1]

In [70]:
df3[::1]

In [71]:
df3[:-1]

In [72]:
df3[:]

In [73]:
df3[:, -1]

In [74]:
df3[: -1]

In [75]:
df3[:]

In [76]:
df3[-1:]

In [77]:
df3[-1:][0]

In [78]:
df3.loc[:, 1]

In [79]:
df3.loc[:, [1]]

In [80]:
df3.iloc[:, [1]]

In [81]:
df3.iloc[:, [1:3]]

In [82]:
df3.iloc[:, :]

In [83]:
df3.iloc[:, :-1]

In [84]:
df3.iloc[:, :-4]

In [85]:
df3.iloc[:, :-3]

In [86]:
df3.iloc[:, :-1]

In [87]:
df3.iloc[:, :-0]

In [88]:
df3.iloc[:, :-2]

In [89]:
df3.iloc[:, :-3]

In [90]:
df3.iloc[:, -3:]

In [91]:
ibre2 = ibre.loc[[1995, 2002, 2015], ['Agropecuária', 'Industria Total', 'Serviços Total']].T

In [92]:
ibre2.index = ['AGR', 'IND', 'SERV']

ibre2.columns = ['95', '2', '15']

In [93]:
ibre2 = ibre.loc[[1995, 2002, 2015], ['Agropecuária', 'Industria Total', 'Serviços Total']].T

In [94]:
ibre2.index = ['AGR', 'IND', 'SERV']

ibre2.columns = ['95', '2', '15']
ibre2

In [95]:
prod = df3.iloc[:, -3:]

In [96]:
prod = df3.iloc[:, -3:]
prod

In [97]:
prod = df3.iloc[:, -3:]
np.array(prod)

In [98]:
prod = df3.iloc[:, -3:]
np.array(prod).reshape(9)

In [99]:
P_D = np.array(ibre2).reshape(9)
P_M = np.array(prod).reshape(9)

In [100]:
prod = df3.iloc[:, -3:]

In [101]:
plt.plot(P_D, P_M)

In [102]:
axx, fig = plt.subplots(1, figsize=(12,8))
ax.scatter(P_D, P_M)

In [103]:
ax, fig = plt.subplots(1, figsize=(12,8))
ax.scatter(P_D, P_M)

In [104]:
fig, ax = plt.subplots(1, figsize=(12,8))
ax.scatter(P_D, P_M)

In [105]:
fig, ax = plt.subplots(1, figsize=(12,8))
ax.scatter(P_D, P_M, s=200, color='red')

In [106]:
P_D

In [107]:
str(AGR)

In [108]:
str('AGR')

In [109]:
'AGR' + str(2)

In [110]:
'AGR_' + str(2)

In [111]:
'AGR_' + str([95, 2, 15])

In [112]:

[]'AGR_' + str(i) for i in range(3) ]

In [113]:

['AGR_' + str(i) for i in range(3) ]

In [114]:
anos2 =  [95, 2, 15]
['AGR_' + str(i) for i in anos ]

In [115]:
anos2 =  [95, 2, 15]
['AGR_' + str(i) for i in anos2 ]

In [116]:
anos2 =  [95, 2, 15]
agr = ['AGR_' + str(i) for i in anos2 ]
ind = ['IND_' + str(i) for i in anos2 ]
serv = ['SERV_' + str(i) for i in anos2 ]

In [117]:
ind

In [118]:
serv

In [119]:
P_D

In [120]:
texto = [agr, ind, serv]

In [121]:
texto = [agr, ind, serv]
texto

In [122]:
texto = [agr, ind, serv].flatten
texto

In [123]:
texto = np.array([agr, ind, serv]).reshape(9)
texto

In [124]:
texto = np.array([agr, ind, serv], dtype=object).reshape(9)
texto

In [125]:
fig, ax = plt.subplots(1, figsize=(12,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(texto[i])
    

In [126]:
fig, ax = plt.subplots(1, figsize=(12,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(P_D[i], P_M[i], texto[i])
    

In [127]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(P_D[i], P_M[i], texto[i])
    

In [128]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=100, color='red')
for i in range(len(texto)):
    ax.text(P_D[i], P_M[i], texto[i])
    

In [129]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(P_D[i]+1, P_M[i], texto[i])
    

In [130]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(P_D[i]+0.5, P_M[i], texto[i])
    

In [131]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(P_D[i]+0.5, P_M[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produtividade (Capital humano) - Modelo', fontsize=20)

In [132]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(P_D[i]+0.5, P_M[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produtividade (Capital humano) - Modelo', fontsize=20)

(m, b) = np.polyfit(P_M, P_D, 1)
yp = np.polyval([m, b], P_M)
ax.plot(P_M, yp, color='black', label='Regression Line')

In [133]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(P_D[i]+0.5, P_M[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produtividade (Capital humano) - Modelo', fontsize=20)

(m, b) = np.polyfit(P_D, P_M 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='black', label='Regression Line')

In [134]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(P_D[i]+0.5, P_M[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produtividade (Capital humano) - Modelo', fontsize=20)

(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='black', label='Regression Line')

In [135]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(P_D[i]+0.5, P_M[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produtividade (Capital humano) - Modelo', fontsize=20)

(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Regression Line')

In [136]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(P_D[i]+0.5, P_M[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produtividade (Capital humano) - Modelo', fontsize=20)

(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Regression Line')
ax.legend(loc="upper left", prop={'size': 18})

In [137]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(P_D[i]+0.5, P_M[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produtividade (Capital humano) - Modelo', fontsize=20)

(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Linha de Regressão')
ax.legend(loc="upper left", prop={'size': 18})

In [138]:
P_D = np.array(ibre2).reshape(9).log 
P_M = np.array(prod).reshape(9)

In [139]:
P_D = np.array(ibre2).reshape(9).log() 
P_M = np.array(prod).reshape(9)

In [140]:
P_D = np.log(np.array(ibre2).reshape(9) )
P_M = np.log(np.array(prod).reshape(9) )

In [141]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(P_D[i]+0.5, P_M[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produtividade (Capital humano) - Modelo', fontsize=20)

(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Linha de Regressão')
ax.legend(loc="upper left", prop={'size': 18})

In [142]:
fig, ax = plt.subplots(1, figsize=(10,8))
ax.scatter(P_D, P_M, s=200, color='red')
for i in range(len(texto)):
    ax.text(P_D[i]+0.05, P_M[i], texto[i])
ax.grid(b=True, which='major', color='k', linestyle=':', lw=.5)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Produtividade - Dados do IBRE', fontsize=20)
ax.set_ylabel('Produtividade (Capital humano) - Modelo', fontsize=20)

(m, b) = np.polyfit(P_D, P_M, 1)
yp = np.polyval([m, b], P_D)
ax.plot(P_D, yp, color='blue', label='Linha de Regressão')
ax.legend(loc="upper left", prop={'size': 18})

In [143]:
df

In [144]:
df2

In [145]:
df2

In [146]:
df2.describe

In [147]:
df2.describe()

In [148]:
df2.T.describe()

In [149]:
df2.T

In [150]:
df2.T

In [151]:
df2.T.describe()

In [152]:
df2.T.describe()[1]

In [153]:
df2.T.describe()

In [154]:
phi = tf.convert_to_tensor(np.array([0.4, 0.5, 0.6]).reshape(1,3) )

In [155]:
phi = tf.convert_to_tensor(np.array([0.4, 0.5, 0.6]).reshape(1,3) )

N_if(d)

In [156]:
phi = tf.convert_to_tensor(np.array([0.4, 0.5, 0.6]).reshape(1,3) )

np.array(N_if(d) )

In [157]:
phi = tf.convert_to_tensor(np.array([0.4, 0.5, 0.6]).reshape(1,3) )

np.array(N_if(d)).sum

In [158]:
phi = tf.convert_to_tensor(np.array([0.4, 0.5, 0.6]).reshape(1,3) )

np.array(N_if(d)).sum()

In [159]:
l = []

for i in range(100):
    phi = tf.convert_to_tensor(np.array([0.4, 0.5, 0.6]).reshape(1,3) ) + i
    l.append(np.array(N_if(d)).sum())

In [160]:
plt.plot(l)

In [161]:
l = []

for i in range(10):
    phi = tf.convert_to_tensor(np.array([0.4, 0.5, 0.6]).reshape(1,3) ) + i
    l.append(np.array(N_if(d)).sum())

In [162]:
plt.plot(l)

In [163]:
l = []
l2 = np.arange(0, 2, 1e-4)

for i in l2:
    phi = tf.convert_to_tensor(np.array([0.4, 0.5, 0.6]).reshape(1,3) ) + i
    l.append(np.array(N_if(d)).sum())

In [164]:
plt.plot(l)

In [165]:
plt.plot(np.arange(0, len(l2), 1), l)

In [166]:
plt.plot(np.arange(0, 2, 1), l)

In [167]:
plt.plot( l)

In [168]:
l = []
l2 = np.arange(0, 2, 1)

for i in l2:
    phi = tf.convert_to_tensor(np.array([0.4, 0.5, 0.6]).reshape(1,3) ) + i
    l.append(np.array(N_if(d)).sum())

In [169]:
plt.plot(l)

In [170]:
l = []
l2 = np.arange(0, 2, 1e-4)

for i in l2:
    phi = tf.convert_to_tensor(np.array([0.4, 0.5, 0.6]).reshape(1,3) ) + i
    l.append(np.array(N_if(d)).sum())

In [171]:
plt.plot(l)

In [172]:
l2

In [173]:
l = []
l2 = np.arange(0, 2, 1e-4)

for i in l2:
    phi = tf.convert_to_tensor(np.array([0.4, 0.5, 0.6]).reshape(1,3) ) + i
    l.append(np.array(N_if(d)).sum())
    phi = tf.convert_to_tensor(np.array([0.4, 0.5, 0.6]).reshape(1,3) )

In [174]:
plt.plot(l)

In [175]:
df2.T.describe()

In [176]:
df2.T.describe()[1]

In [177]:
df2.T.describe()

In [178]:
df2.T.describe().loc[[1], :]

In [179]:
df2.T.describe().loc[[1], 1]

In [180]:
df2.T.describe().loc[[1], [1]]

In [181]:
df2.T.describe().iloc[[1], [1]]

In [182]:
df2.T.describe().iloc[[1], ]

In [183]:
df2.T.describe().iloc[[1], :]

In [184]:
np.array(df2.T.describe().iloc[[1], :] )

In [185]:
np.array(df2.T.describe().iloc[[1], :] ).reshape(1,3)

In [186]:
tf.convert_to_tensor(np.array(df2.T.describe().iloc[[1], :] ).reshape(1,3) )

In [187]:
pars(15)
#alfa = tf.convert_to_tensor(np.array([0.333, 0.334, 0.333]).reshape(1,3) )

phi = tf.convert_to_tensor(np.array(df2.T.describe().iloc[[1], :] ).reshape(1,3) )

In [188]:


Bd2 =   ((-10.0, 10.0), )*st 

#Bd = (tf.constant(-20.0, dtype=tf.float64), tf.constant(20.0, dtype=tf.float64), )*st

Bd2 = np.array(Bd2, dtype=object)

In [189]:
sol = differential_evolution(tflow, Bd2, strategy='best1exp',
                             tol=0,
                             polish=True, 
                             popsize=100, 
                             maxiter=400,
                             recombination =0.7,
                             disp=True)

#3.95073768,   8.75856475, -17.4818774 

In [190]:
sol.x

In [191]:
Bd2

In [192]:
sol

In [193]:
sol = differential_evolution(tflow, Bd2, strategy='best1exp',
                             tol=0,
                             polish=True, 
                             popsize=100, 
                             maxiter=400,
                             recombination =0.7,
                             disp=True)

#array([ 7.53593756, -8.77005193,  6.95345626])

In [194]:
sol

In [195]:
phi

In [196]:
sol = differential_evolution(tflow, Bd2, strategy='best1exp',
                             tol=0,
                             polish=True, 
                             popsize=100, 
                             maxiter=400,
                             recombination =0.8,
                             disp=True)

#array([ 7.53593756, -8.77005193,  6.95345626])
#array([-9.53593756, -8.77005193,  6.95345626])   3.69566e-62

In [197]:
sol

In [198]:
sol = differential_evolution(tflow, Bd2, strategy='best1exp',
                             tol=0,
                             polish=True, 
                             popsize=100, 
                             maxiter=400,
                             recombination =0.8,
                             disp=True)


# 7.53593756, -8.77005193,  6.95345626          3.695661944942129e-62

In [199]:
from matplotlib import rc, animation
%matplotlib inline
rc('animation', html='html5')    

In [200]:
<nav>
<a href=”https://www.google.com"> LinkedIn </a> |
<a href=”/css/”> Github </a> |
<a href=”/js/”> Médio </a> |
</nav>